# 02-04 : Sentiment Analysis

reference: https://huggingface.co/spaces/yangheng/Multilingual-Aspect-Based-Sentiment-Analysis

In [1]:
import pandas as pd
from pyabsa import ABSAInstruction, AspectTermExtraction, AspectPolarityClassification

/bin/sh: 1: nvidia-smi: not found


No CUDA GPU found in your device
[2023-08-02 06:14:40] (2.3.1) PyABSA(2.3.1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.16.27

[New Feature] Aspect Sentiment Triplet Extraction since v2.1.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_sentiment_triplet_extration)
[New Feature] Aspect CategoryOpinion Sentiment Quadruple Extraction since v2.2.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_opinion_sentiment_category_extraction)



/usr/local/lib/python3.11/multiprocessing/pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


## Data Load

In [2]:
df_vodacom = pd.read_parquet('../../data/interim/02-03_twitter_vodacom.parquet')

# display the first 3 rows
print(df_vodacom.shape)
display(df_vodacom.head(3))

(983, 22)


,id,conversationId,date,rawContent,inReplyToTweetId,inReplyToUserId,inReplyToUsername,inReplyToDisplayName,language,likeCount,...,retweetCount,source,url,userCreated,userDisplayName,userFollowersCount,userId,userLocation,userStatusesCount,userUsername
0,1684937340169375744,1684937340169375744,2023-07-28 14:42:21+00:00,"Serious question:\n\nHas anyone, anywhere ever...",None,None,None,None,en,2,...,1,Twitter for Android,https://twitter.com/TopEditorInt/status/168493...,2009-03-04 11:44:38+00:00,TopEditor Internatio,2956,22759776,USA & SA & UK,70294,TopEditorInt
1,1684892139375710208,1684890636997029888,2023-07-28 11:42:44+00:00,@Vodacom That’s why I wanted to talk to someon...,1684891762496565248,14574763,Vodacom,Vodacom,en,0,...,0,Twitter for iPhone,https://twitter.com/AkonaMhlana/status/1684892...,2009-05-20 14:38:05+00:00,Ntaba ayilali👏🏽,135,41370409,South Africa,866,AkonaMhlana
2,1684890636997029888,1684890636997029888,2023-07-28 11:36:46+00:00,@Vodacom what number should I dial to talk to...,None,14574763,Vodacom,Vodacom,en,0,...,0,Twitter for iPhone,https://twitter.com/AkonaMhlana/status/1684890...,2009-05-20 14:38:05+00:00,Ntaba ayilali👏🏽,135,41370409,South Africa,866,AkonaMhlana


In [3]:
df_vodafone = pd.read_parquet('../../data/interim/02-03_twitter_vodafone.parquet')

# display the first 3 rows
print(df_vodafone.shape)
display(df_vodafone.head(3))

(1482, 22)


,id,conversationId,date,rawContent,inReplyToTweetId,inReplyToUserId,inReplyToUsername,inReplyToDisplayName,language,likeCount,...,retweetCount,source,url,userCreated,userDisplayName,userFollowersCount,userId,userLocation,userStatusesCount,userUsername
0,1686000773698654210,1686000773698654210,2023-07-31 13:08:03+00:00,Tried customer service. No good. Tried Tobi -...,None,None,None,None,en,0,...,0,Twitter for Android,https://twitter.com/JohndeWinton/status/168600...,2011-09-23 08:59:27+00:00,John de Winton💙,75,378488745,Wales,3835,JohndeWinton
1,1685735867090964480,1685735867090964480,2023-07-30 19:35:24+00:00,@VodafoneUK Your Tobi chat ‘help’ service is t...,None,20678384,VodafoneUK,Vodafone UK,en,5,...,0,Twitter for iPhone,https://twitter.com/garry__spence/status/16857...,2012-08-06 00:33:55+00:00,Garry Spence,25737,739523767,"Scotland, United Kingdom",10492,garry__spence
2,1685556031110144000,1685556031110144000,2023-07-30 07:40:48+00:00,@VodafoneUK @Ofcom Vodafone seem completely un...,None,20678384,VodafoneUK,Vodafone UK,en,1,...,0,Twitter for iPhone,https://twitter.com/vincentconnolly/status/168...,2011-05-21 08:55:53+00:00,AMUdoc 💙,1034,302499728,,4164,vincentconnolly


## Test PyABSA

In [8]:
test_samples = [
    df_vodacom.iloc[0].rawContent,
    df_vodacom.iloc[44].rawContent,
    df_vodacom.iloc[124].rawContent,
    df_vodacom.iloc[150].rawContent,
    df_vodacom.iloc[200].rawContent,
    df_vodacom.iloc[300].rawContent,
    
    df_vodafone.iloc[0].rawContent,
    df_vodafone.iloc[44].rawContent,
    df_vodafone.iloc[124].rawContent,
    df_vodafone.iloc[150].rawContent,    
    df_vodafone.iloc[200].rawContent,
    df_vodafone.iloc[300].rawContent,    
]


test_samples

['Serious question:\n\nHas anyone, anywhere ever had a positive or fulfilling experience with a chat bot such as @FNBSA "Secure Chat" or @Vodacom Tobi or the like?',
 "@Vodacom111  \nMy airtime is disappearing, but you r saying there's no active subscription...I think I've tried several times to get help from you but Dololo.\nOr do I've to deactivate this number because of your poor services. \n\nPlease don't me about your TOBI 🚮🤮 https://t.co/YwwJa7sFzx",
 '@Vodacom your TOBI is useless. Can an agent please call me on 083 408 5621 urgently',
 '@Vodacom111 TOBi Bot is not really helping me resolve my query, I need to speak to an operator. https://t.co/OO9f0qWeCE',
 '@Vodacom this is fucking bullshit, selling data that expires with a shity network. Phoning your call centre is fucking waste of time with stupid Tobi. https://t.co/iC6qO1YHhD',
 '@Vodacom111 is there a human behind this account???! IM JAS FOR FUCKING TOBI AND HIS DOM POES.... excuse my French',
 "Tried customer service.  No

In [9]:
quadruple_extractor = ABSAInstruction.ABSAGenerator('multilingual')

[2023-08-02 06:19:02] (2.3.1) Downloading checkpoint:multilingual 
[2023-08-02 06:19:02] (2.3.1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2023-08-02 06:19:02] (2.3.1) Checkpoint already downloaded, skip


In [10]:
quadruples = [quadruple_extractor.predict(text=text, max_length=512) for text in test_samples]

{'text': 'Serious question:\n\nHas anyone, anywhere ever had a positive or fulfilling experience with a chat bot such as @FNBSA "Secure Chat" or @Vodacom Tobi or the like?', 'Quadruples': [{'aspect': 'chat bot', 'polarity': 'negative', 'opinion': 'positive or fulfilling', 'category': 'SOFTWARE#GENERAL'}]}
{'text': "@Vodacom111  \nMy airtime is disappearing, but you r saying there's no active subscription...I think I've tried several times to get help from you but Dololo.\nOr do I've to deactivate this number because of your poor services. \n\nPlease don't me about your TOBI 🚮🤮 https://t.co/YwwJa7sFzx", 'Quadruples': [{'aspect': 'airtime', 'polarity': 'negative', 'opinion': 'disappearing', 'category': 'SERVICE#GENERAL'}, {'aspect': 'services', 'polarity': 'negative', 'opinion': 'poor', 'category': 'SERVICE#GENERAL'}]}
{'text': '@Vodacom your TOBI is useless. Can an agent please call me on 083 408 5621 urgently', 'Quadruples': [{'aspect': 'TOBI', 'polarity': 'negative', 'opinion': 'usele

In [11]:
quadruples

[{'text': 'Serious question:\n\nHas anyone, anywhere ever had a positive or fulfilling experience with a chat bot such as @FNBSA "Secure Chat" or @Vodacom Tobi or the like?',
  'Quadruples': [{'aspect': 'chat bot',
    'polarity': 'negative',
    'opinion': 'positive or fulfilling',
    'category': 'SOFTWARE#GENERAL'}]},
 {'text': "@Vodacom111  \nMy airtime is disappearing, but you r saying there's no active subscription...I think I've tried several times to get help from you but Dololo.\nOr do I've to deactivate this number because of your poor services. \n\nPlease don't me about your TOBI 🚮🤮 https://t.co/YwwJa7sFzx",
  'Quadruples': [{'aspect': 'airtime',
    'polarity': 'negative',
    'opinion': 'disappearing',
    'category': 'SERVICE#GENERAL'},
   {'aspect': 'services',
    'polarity': 'negative',
    'opinion': 'poor',
    'category': 'SERVICE#GENERAL'}]},
 {'text': '@Vodacom your TOBI is useless. Can an agent please call me on 083 408 5621 urgently',
  'Quadruples': [{'aspect':